In [1]:
import torch
from torchvision import datasets, models, transforms
from torch.nn import Linear

## 超参数

In [2]:
batch_size = 64
k_fold = 10
cloud_num = 28
focal_loss_gmma = 2.0
epochs = 100
model_name = 'resnet-50'

## 模型加载

使用`torchvision.models`中的resnet50作为预训练模型，并修改其全连接层。

In [3]:
def load_model(cloud_class_nums):
    pretrained_model = models.resnet50(weights=models.resnet.ResNet50_Weights.DEFAULT)
    num_ftrs = pretrained_model.fc.in_features
    pretrained_model.fc = Linear(num_ftrs, cloud_class_nums)
    return pretrained_model

model = load_model(cloud_num)

## 构建数据集类
根据`torch`官方文档修正`transform`参数:
- 训练阶段:将图片随机切分并`resize`到$224 \times 224$
- 验证阶段:
  - 将图片`resize`到$256 \times 256$
  - 截取`center`的$24 \times 224$

In [4]:
import os
from torch.utils.data import Dataset
from PIL import Image

class CloudImageDataSet(Dataset):
    def __init__(self, img_dir_path, img_set_with_label: dict, data_transforms=None, dataset = ''):
        self.imgs = list(img_set_with_label.keys())
        self.labels = list(img_set_with_label.values())
        self.data_transforms = data_transforms
        self.img_dir_path = img_dir_path
        self.dataset = dataset
        
    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self, k):
        img_name = self.imgs[k]
        label = torch.nn.functional.one_hot(torch.tensor(self.labels[k] - 1), num_classes=cloud_num).double()
        img_path = os.path.join(self.img_dir_path, img_name)
        with open(img_path, 'rb') as img_file:
            img = Image.open(img_file)
            img = img.convert('RGB')
        img_file.close()
        return self.data_transforms[self.dataset](img), label

    
data_transforms = {
    'Train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'Test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
import json
import pandas

train_dataset_df = pandas.read_csv('/home/codefor/work/cloud_dataset/train.csv').T
train_image_name_and_label_set = {}
for k, v in train_dataset_df.to_dict().items():
    train_image_name_and_label_set[v['FileName']] = v['Code']
val_dataset_df = pandas.read_csv('/home/codefor/work/cloud_dataset/standard_answer.csv').T
val_image_name_and_label_set = {}
for k, v in val_dataset_df.to_dict().items():
    val_image_name_and_label_set[v['FileName']] = v['Code']

In [7]:
model = model.to(device)

## Focal Loss

$-(1-p_t)log(p_t)$

In [8]:
from torch.nn import Softmax, CrossEntropyLoss

class FocalLoss(torch.nn.Module):
    def __init__(self, gmma):
        super().__init__()
        self.gmma = gmma
        self.softmax = Softmax(dim=1)
        self.cross_entropy_loss = CrossEntropyLoss()
    
    def forward(self, pred, label):
        pred_after_softmax = self.softmax(pred)
        ce = - torch.log(pred_after_softmax) * label
        focal_loss = (((1 - pred_after_softmax * label) ** self.gmma) * ce).sum() / batch_size
        return focal_loss

In [9]:
# loss_fn = torch.nn.CrossEntropyLoss()
loss_fn = FocalLoss(gmma=focal_loss_gmma)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
train_dataset = CloudImageDataSet('/home/codefor/work/cloud_dataset/images/', train_image_name_and_label_set, data_transforms, 'Train')
val_dataset = CloudImageDataSet('/home/codefor/work/cloud_dataset/images', val_image_name_and_label_set, data_transforms, 'Test')

## 训练
将`test.csv`中的数据加载，进行`K-Fold`训练

In [ ]:
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
import time

# def k_fold_split(dataset, epoch, total_size, seg):
#     trll = 0
#     trlr = epoch * seg
#     vall = trlr
#     valr = epoch * seg + seg
#     trrl = valr
#     trrr = total_size
#     train_left_indices = list(range(trll,trlr))
#     train_right_indices = list(range(trrl,trrr))       
#     train_indices = train_left_indices + train_right_indices
#     val_indices = list(range(vall,valr))
#     train_set = torch.utils.data.dataset.Subset(dataset, train_indices)
#     val_set = torch.utils.data.dataset.Subset(dataset, val_indices)
#     return train_set, val_set


# total_size = len(dataset)
# fraction = 1 / k_fold
# seg = int(total_size * fraction)
max_f1_score = 0
max_f1_score_model_name = ''

for epoch in range(epochs):
    # train_set, val_set = k_fold_split(dataset, epoch % k_fold, total_size, seg)
    train_set, val_set = train_dataset, val_dataset
    train_dataloader =  DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8)
    val_dataloader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=8)
    size = len(train_dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(train_dataloader):
        X = X.to(device)
        y = y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 20 == 0:
            loss, current = loss.item(), batch * batch_size
    test_loss, correct = 0, 0
    size = len(val_dataloader.dataset)
    num_batches = len(val_dataloader)
    model.eval()
    with torch.no_grad():
        pred_set = torch.tensor([])
        y_set = torch.tensor([])
        for X, y in val_dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            pred_set = torch.cat((pred_set, pred.cpu()), 0)
            y_set = torch.cat((y_set, y.cpu()), 0)
            correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    f1 = f1_score(pred_set.argmax(1), y_set.argmax(1), average=None).sum().item() / cloud_num
    if f1 > max_f1_score:
        max_f1_score = f1
        max_f1_score_model_name = f'./model_state_dict/model-{batch_size}-{focal_loss_gmma}-{model_name}-epoch{epoch}-f1-score{f1}'
    torch.save(model.state_dict(), f'./model_state_dict/model-{batch_size}-{focal_loss_gmma}-{model_name}-epoch{epoch}-f1-score{f1}')
    log_file = open(f'./log/log-{batch_size}-{focal_loss_gmma}-{model_name}.txt', 'a')
    print(f'epoch {epoch}---------------------------', file=log_file)
    print(f"Val: \n Accuracy: {(100*correct):>0.1f}%, f1_score: {f1:>8f} Avg loss: {test_loss:>8f} \n", file=log_file)
    log_file.close()

In [19]:
test_dataset_df = pandas.read_csv('/home/codefor/work/cloud_dataset/test.csv').T
test_image_name_set = []
for k, v in test_dataset_df.to_dict().items():
    test_image_name_set.append(v['FileName'])

In [20]:
class CloudImageDataSetForTest(Dataset):
    def __init__(self, img_dir_path, img_set: list, data_transforms=None, dataset = ''):
        self.imgs = img_set
        self.data_transforms = data_transforms
        self.img_dir_path = img_dir_path
        self.dataset = dataset
        
    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self, k):
        img_name = self.imgs[k]
        img_path = os.path.join(self.img_dir_path, img_name)
        with open(img_path, 'rb') as img_file:
            img = Image.open(img_file)
            img = img.convert('RGB')
        img_file.close()
        return self.data_transforms[self.dataset](img)

In [21]:
test_dataset = CloudImageDataSetForTest('/home/codefor/work/cloud_dataset/images/', test_image_name_set, data_transforms, 'Test')
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8)

In [22]:
model.load_state_dict(torch.load(max_f1_score_model_name))

with torch.no_grad():
    model.eval()
    pred_set = torch.tensor([])
    for X in test_dataloader:
        X = X.to(device)
        pred = model(X)
        pred_set = torch.cat((pred_set, pred.cpu().argmax(1) + 1), 0)

In [23]:
submission = zip(test_image_name_set, pred_set.int().tolist())
submission = dict(submission)
submission = pandas.DataFrame([submission]).T
submission.to_csv('./cloud_dataset/submission.csv')

In [24]:
f1

0.374368257841779

In [25]:
test_loss, correct = 0, 0
size = len(val_dataloader.dataset)
num_batches = len(val_dataloader)
model.eval()
with torch.no_grad():
    pred_set = torch.tensor([])
    y_set = torch.tensor([])
    for X, y in val_dataloader:
        X = X.to(device)
        y = y.to(device)
        pred = model(X)
        test_loss += loss_fn(pred, y).item()
        pred_set = torch.cat((pred_set, pred.cpu()), 0)
        y_set = torch.cat((y_set, y.cpu()), 0)
        correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
test_loss /= num_batches
correct /= size
f1 = f1_score(pred_set.argmax(1), y_set.argmax(1), average=None).sum().item() / cloud_num
torch.save(model.state_dict(), f'./model_state_dict/{model_name}-epoch{epoch + 51}-f1{f1}')
print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, f1_score: {f1:>8f} Avg loss: {test_loss:>8f} \n")

Test Error: 
 Accuracy: 52.1%, f1_score: 0.374368 Avg loss: 0.997961 



In [ ]:
model.load_state_dict(torch.load(max_f1_score_model_name))